---
title: $k$-Means Clustering
jupyter: python3
---

In [ ]:
#| echo: false
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Victorian England


## 1854 Cholera Outbreak

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/jupyter_notebooks/06-Clustering-I-kmeans.ipynb)

:::: {.columns }

::: {.column width="40%"}
![London cholera outbreak](./figs/Punch-A_Court_for_King_Cholera.png){width=400}
^[Public Domain, https://commons.wikimedia.org/w/index.php?curid=680455]
:::

::: {.column width="60%" .incremental}
- There was a horrific cholera outbreak in **1854** Soho, London.
- Common wisdom at the time was that disease spread by breathing "foul air" (miasma).  
- The London sewer system had not yet reached Soho.  
- Most homes had cesspits under the floor, which often overflowed.
- "Night Soil Men" would regularly collect and sell to farmers or dump in the Thames.
:::

::::

## John Snow

:::: {.columns }

::: {.column width="60%" .incremental}
- John Snow, a local physician, extensively studied the patterns of illness across Soho due to cholera.
- In the course of his studies, his attention was drawn to one neighborhood around Broad Street.
- In 10 days, 500 people in the area died. 
:::

::: {.column width="40%"}
![John Snow](./figs/L06-John-Snow-Portrait.png)
:::

::::

## John's Snow Map

:::: {.columns }

::: {.column width="60%"}
![London cholera outbreak](figs/L6-Snow-cholera-map-1.png)
:::

::: {.column width="40%" .incremental}
- In uncovering the source of this outbreak, Snow prepared this map.
- From this map he could clearly see the deaths were clustered around an area.
- The neighborhood was all served by a water pump on Broad St.
- The pump handle was removed and illnesses decreased dramatically.

::: {.fragment}
![Broad St. water pump](./figs/L6-John_Snow_memorial_and_pub.png){width=180}
:::

:::

::::

---

:::: {.columns}
::: {.column width="50%"}
![J. Snow Book](figs/L06-snow-book-cover.png){height=500}
:::
::: {.column width="50%" .incremental}
- He later published his results^[By John Snow - Published by C.F. Cheffins, Lith, Southhampton Buildings, London, England, 1854 in Snow, John. On the Mode of Communication of Cholera, 2nd Ed, John Churchill, New Burlington Street, London, England, 1855.]
- Results from the cluster map.
- Results from a double blind study of two neighborhoods drawing water upriver and downriver of the polluted portion of the Thames.
- Other anecdotes of visitors to Soho, etc.
:::
::::

::: {.content-hidden when-profile="slides"}
## References and Further Reading 

Images and information taken from [wikipedia](https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak), 
[National Library of Medicine](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7150208/) and the 
[Internet Archive](https://archive.org/details/b28985266/page/n3/mode/2up) and
[MSU's John Snow Archive](https://johnsnow.matrix.msu.edu/).

John Snow's original data is recreated [here](https://blog.rtwilson.com/john-snows-cholera-data-in-more-formats/).
:::

# Clustering

## Clustering

::: {.fragment}
Clustering is a very important way of discovering __structure__ in data.
:::

::: {.fragment}
It is so important because it is __common__ for data to show clusters.
:::

::: {.incremental}
* Locations where millionaires live
* The number of hours people work each week
* Demographics ("soccer parents", "bored retirees", "unemployed millenials", etc)
:::

::: {.content-hidden when-profile="web"}
## Clustering Continued
:::

::: {.incremental}
* We can often simplify or compress our data if we recognize the existence of clusters.
* Further, we can often interpret clusters by assigning them labels.
* However, note that these categories or "labels" are assigned __after__ the fact.
* And, we may __not__ be able to interpret clusters or assign them labels in some cases.
* That is, clustering represents the first example we will see of __unsupervised__ learning.
:::

## Supervised vs Unsupervised

__Supervised__ methods:  Data items have labels, and we want to learn a function that correctly assigns labels to new data items.

__Unsupervised__ methods:  Data items do not have labels, and we want to learn a function that extracts  important patterns from the data.

## Applications of Clustering

* Image Processing
    * Cluster images based on their visual content.
    * Compress images based on color clusters.
* Web Mining
    * Cluster groups of users based on webpage access patterns.
    * Cluster web pages based on their content.
* Bioinformatics
    * Cluster similar proteins together (by structure or function).
    * Cluster cell types (by gene activity).
* And many more ...

## The Clustering Problem

When we apply clustering, what problem are we trying to solve?

::: {.fragment}
We will answer this question informally at first. 
:::

::: {.fragment}
(But soon we will look at formal criteria!)
:::

::: {.fragment .fade-in}
Informally, a __clustering__ is:
    
> a grouping of data objects, such that the objects within a group are similar (or near) to one another and dissimilar (or far) from the objects in other groups.

(keep in mind that if we use a distance function as a dissimilarity measure, then "far" implies "different")
:::

::: {.content-hidden when-profile="web"}
## The Clustering Problem, continued
:::

![](figs/L06-clustering-1.png){width=350 fig-align="center"}
    
So we want our clustering algorithm to:

* <font color = "blue">minimize</font> intra-cluster distances.
* <font color = "red">maximize</font> inter-cluster distances.


::: {.content-hidden when-profile="web"}
## The Clustering Problem, continued

![](figs/L06-clustering-1.png){width=350 fig-align="center"}
:::
Here are the basic questions we need to ask about clustering:

* What is the right kind of <font color = "blue">"similarity"</font> to use?
* What is a <font color="blue">*good* partition</font> of objects?
    * i.e., how is the quality of a solution measured?
* <font color = "blue">How to find</font> a good partition?
    * Are there efficient algorithms?  
    * Are there algorithms that are guaranteed to find good clusters?

::: {.content-hidden when-profile="web"}
## The Clustering Problem, continued
:::

Now note that even with our more-formal discussion, the criteria for deciding on
a "best" clustering can still be ambiguous.
    
![](figs/L06-clustering-2.png){width=600 fig-align="center"}

To accommodate the ambiguity here, one approach is to seek a __hierarchical__ clustering.

That is, as set of nested clusters organized in a tree.

We'll discuss hierarchical cluster in an upcoming lecture.

---

For today, we'll focus on __partitional__ clustering. 

In a partitional clustering, the points are divided into a set of __non-overlapping__ groups.
    
![](figs/L06-partitional-clustering.png){width=600 fig-align="center"}
    
In a partitional clustering.

* Each object belongs to __one__, and only one, cluster.
* The set of clusters covers all the objects.

We are going to assume for now that the number of clusters is given in advance.

We will denote the number of clusters as $k$.

# The $k$-means Algorithm

## Assumptions

Now, we are ready to state our first formalization of the clustering problem.

We will assume that 

* data items are represented by points in $d$-dimensional space, $\mathbb{R}^d$, i.e.,  has $d$ features,
* the number of points, $N$, is given, and
* the number of clusters $K$ is given.

## Minimizing a Cost Function

Find $K$ disjoint clusters $C_k$, each described by points $c_1, \dots, c_K$
(called <font color="blue"> _centers_, _centroids_,</font> or <font color = "blue"> _means_)</font>
that minimizes

$$
\sum_{k=1}^K \sum_{x\in C_k} \Vert x-c_k\Vert^2. 
$$

The literature sometimes calls this the _Inertia_ of the clustering.

::: aside
See [Norms](./05-Distances-Timeseries.html#norms) in
[Distances and Timeseries](./05-Distances-Timeseries.html) for a refresher.
:::

---

We now have a <font color="blue">formal</font> definition of a clustering.

This is not the only definition possible, but it is an intuitive and simple one.

How hard is it to solve this problem?

* $k=1$ and $k=n$ are easy special cases. Why?
* But, this problem is __NP-hard__ if the dimension of the data is at least 2.
    * We don't expect that there is any exact, efficient algorithm in general.

__Nonetheless,__ there is a simple algorithm that works quite well in practice!

::: {.content-hidden when-profile="slides"}
## $k$-means as NP-hard

For context, **NP-hard** is a term used in the study of
[computational complexity](https://en.wikipedia.org/wiki/Computational_complexity_theory).

Problems in [**P**](https://en.wikipedia.org/wiki/P_(complexity)) are those that can be solved in polynomial time.

Problems in [**NP**](https://en.wikipedia.org/wiki/NP_(complexity)) are those that
can be _verified_ in polynomial time but not necessarily _solved_ in polynomial time.

[**NP-hard**](https://en.wikipedia.org/wiki/NP-hardness) problems are those that are at least as hard as the hardest problems in **NP**.

[**NP-complete**](https://en.wikipedia.org/wiki/NP-complete) problems are those that are both NP-hard and in NP.

You can prove that the $k$-means problem is NP-hard by finding a reduction from a known NP-hard problem such as the [**Partition Problem**](https://en.wikipedia.org/wiki/Partition_problem).

::: {.callout-note}
The $k$-means problem is NP-hard.
:::

:::

## The $k$-means Algorithm

:::: {.columns }

::: {.column width="40%"}
![](figs/L06-top-ten-algorithms-cover.png){width=300 fig-align="center"}
:::

::: {.column width="60%"}
* There is a "classic" algorithm for this problem.
* It was voted among the __top-10 algorithms__ in data mining!^[As determined at the 2006 IEEE International Conference on Data Mining]
* It is such a good idea that it has been independently discovered multiple times.
* It was first discovered by Lloyd in 1957, so it is often called Lloyd's algorithm.
* It is called the "$k$-means algorithm."
* Not to be confused with the $k$-means problem of which this is a heuristic solution.
:::

::::

::: aside
The other from the top 10 were SVM, Apriori, EM, PageRank, AdaBoost, kNN, Naive Bayes, and CART.
:::

::: {.content-hidden when-profile="web"}
## The $k$-means algorithm
:::

1. Pick $K$ cluster centers $\{c_1, \dots, c_K\}$.  
    - These can be randomly chosen data points, or by some other method such as
     [$k$-means++](https://en.wikipedia.org/wiki/K-means%2B%2B)
2. For each $j$, define the cluster $C_j$ as the set of points in $X$ that are <font color="blue">closest to center</font> $c_j$.  
    - Nearer to $c_j$ than to any other center.
3. For each $j$, update $c_j$ to be <font color="blue">the center of mass of cluster</font> $C_j$.  
    - In other words, $c_j$ is the mean of the vectors in $C_j$.
4. Repeat (i.e., go to Step 2) until convergence.
    - Either the cluster centers change below a threshold, or inertia changes below a threshold or a maximum number of iterations is reached.

---

Let's see this in practice with well separated clusters and also look at the Within-Cluster Sum of Square (WCSS).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 300
X, _ = make_blobs(n_samples=n_samples, centers=3, cluster_std=0.60, random_state=42)

# Initialize centroids randomly
k = 3
centroids = X[np.random.choice(n_samples, k, replace=False)]

# Function to assign points to clusters
def assign_clusters(X, centroids):
    distances = np.sqrt(((X - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

# Function to update centroids
def update_centroids(X, labels, k):
    return np.array([X[labels == i].mean(axis=0) for i in range(k)])

# Function to calculate within-cluster sum of squares
def calculate_wcss(X, labels, centroids):
    return sum(np.sum((X[labels == i] - centroids[i])**2) for i in range(k))

# Set up the clustering progress plot
fig1, axs = plt.subplots(2, 3, figsize=(10, 6))
axs = axs.ravel()

# Colors for each cluster
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# List to store WCSS values
wcss_values = []

# Run k-means iterations and plot
for i in range(6):
    # Assign clusters
    labels = assign_clusters(X, centroids)
    
    # Calculate WCSS
    wcss = calculate_wcss(X, labels, centroids)
    wcss_values.append(wcss)
    
    # Plot the current clustering state
    axs[i].scatter(X[:, 0], X[:, 1], c=[colors[l] for l in labels], alpha=0.6)
    axs[i].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    axs[i].set_title(f'Iteration {i if i < 5 else "Final"}, WCSS: {wcss:.2f}')
    axs[i].set_xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
    axs[i].set_ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)
    
    # Update centroids (except for the last iteration)
    if i < 5:
        centroids = update_centroids(X, labels, k)

plt.tight_layout()

# Create a separate plot for WCSS progress
fig2, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(6), wcss_values, marker='o')
ax.set_title('WCSS Progress')
ax.set_xlabel('Iteration')
ax.set_ylabel('Within-Cluster Sum of Squares')
ax.grid(True)

plt.tight_layout()
plt.show()

---

Let's see this in practice with overlapping clusters and also look at the WCSS.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Set random seed for reproducibility
np.random.seed(42)

# Generate sample data
n_samples = 300
X, _ = make_blobs(n_samples=n_samples, centers=3, cluster_std=3.00, random_state=2)

# Initialize centroids randomly
k = 3
centroids = X[np.random.choice(n_samples, k, replace=False)]

# Function to assign points to clusters
def assign_clusters(X, centroids):
    distances = np.sqrt(((X - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

# Function to update centroids
def update_centroids(X, labels, k):
    return np.array([X[labels == i].mean(axis=0) for i in range(k)])

# Function to calculate within-cluster sum of squares
def calculate_wcss(X, labels, centroids):
    return sum(np.sum((X[labels == i] - centroids[i])**2) for i in range(k))

# Set up the clustering progress plot
fig1, axs = plt.subplots(2, 3, figsize=(10, 6))
axs = axs.ravel()

# Colors for each cluster
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# List to store WCSS values
wcss_values = []

# Run k-means iterations and plot
for i in range(6):
    # Assign clusters
    labels = assign_clusters(X, centroids)
    
    # Calculate WCSS
    wcss = calculate_wcss(X, labels, centroids)
    wcss_values.append(wcss)
    
    # Plot the current clustering state
    axs[i].scatter(X[:, 0], X[:, 1], c=[colors[l] for l in labels], alpha=0.6)
    axs[i].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    axs[i].set_title(f'Iteration {i if i < 5 else "Final"}, WCSS: {wcss:.2f}')
    axs[i].set_xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
    axs[i].set_ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)
    
    # Update centroids (except for the last iteration)
    if i < 5:
        centroids = update_centroids(X, labels, k)

plt.tight_layout()

# Create a separate plot for WCSS progress
fig2, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(6), wcss_values, marker='o')
ax.set_title('WCSS Progress')
ax.set_xlabel('Iteration')
ax.set_ylabel('Within-Cluster Sum of Squares')
ax.grid(True)

plt.tight_layout()
plt.show()

---


## Limitations of $k$-means

As you can see, $k$-means can work very well.

However, we don't have any guarantees on the performance of $k$-means.

In particular, there are various settings in which $k$-means can fail to do a good job.

---

1. __$k$-means tries to find spherical clusters__.

Because each point is assigned to its closest center, the points in a cluster are implicitly assumed to be arranged in a sphere around the center.

In [ ]:
# Author: Phil Roth <mr.phil.roth@gmail.com>
#         Arturo Amor <david-arturo.amor-quiroz@inria.fr>
# License: BSD 3 clause
#
# From https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html

import numpy as np
from sklearn.datasets import make_blobs, make_moons

n_samples = 1500
random_state = 170
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]

X, y = make_blobs(n_samples=n_samples, random_state=random_state)
X_aniso = np.dot(X, transformation)  # Anisotropic blobs
X_varied, y_varied = make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
    )  # Unequal variance
X_filtered = np.vstack(
    (X[y == 0][:500], X[y == 1][:100], X[y == 2][:10])
    )  # Unevenly sized blobs
y_filtered = [0] * 500 + [1] * 100 + [2] * 10

# Generate two half-moon clusters
X_moons, y_moons_true = make_moons(n_samples=200, noise=0.1, random_state=42)

In [ ]:
#| fig-cap: "K-means clustering on a dataset with anisotropic clusters."
#| fig-align: "center"

# Run above cell first

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Perform k-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
y_moons_pred = kmeans.fit_predict(X_moons)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot ground truth
ax1.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons_true, cmap='viridis')
ax1.set_title('Ground Truth')
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')

# Plot k-means clustering result
ax2.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons_pred, cmap='viridis')
ax2.set_title('K-means Clustering')
ax2.set_xlabel('Feature 1')
ax2.set_ylabel('Feature 2')

# Add cluster centers to the k-means plot
ax2.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

plt.tight_layout()
plt.show()

---

In [ ]:
#| fig-cap: "K-means clustering on a dataset with anisotropic gaussian clusters."

# run previous two cells first

common_params = {
    "n_init": "auto",
    "random_state": random_state,
}

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_aniso[:, 0], X_aniso[:, 1], c=y)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_aniso)
axs[1].scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

---

2. __$k$-means tries to find equal-sized clusters.__

For the same reason, $k$-means tends to try to balance the sizes of the clusters.

In [ ]:
#| fig-cap: "K-means clustering on a dataset with unequal-sized clusters."

# run previous three cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_filtered)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_filtered)
axs[1].scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

---

3. __$k$-means is sensitive to variance of the clusters.__


In [ ]:
#| fig-cap: "K-means clustering on a dataset with unequal variance clusters."

# run previous four cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X_varied[:, 0], X_varied[:, 1], c=y_varied)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=3, **common_params)
y_pred = kmeans.fit_predict(X_varied)
axs[1].scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

# plt.suptitle("Unexpected KMeans clusters").set_y(0.95)
plt.show()

---

4. __$k$-means is sensitive to the starting cluster centers.__

If the initial guess (Step 1) is a bad one, $k$-means may get "stuck" in a bad solution.
    
![](figs/L06-kmeans-bad-initialization.png){width=600 fig-align="center"}

---

5. __$k$-means is sensitive to the number of clusters.__

In [ ]:
#| fig-cap: "K-means clustering on a dataset with incorrect number of clusters parameter."

# run previous five cells first

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

axs[0].scatter(X[:, 0], X[:, 1], c=y)
axs[0].set_title("Ground Truth")

kmeans = KMeans(n_clusters=2, **common_params)
y_pred = kmeans.fit_predict(X)
axs[1].scatter(X[:, 0], X[:, 1], c=y_pred)
axs[1].set_title("K-means Clustering")

# Add cluster centers to the k-means plot
axs[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)

plt.show()

## Choosing a Good Initialization

* How can we avoid the kind of bad initialization we just saw?
* A good strategy is to pick points that are distant to each other.
* This strategy is called ["$k$-means++"](https://en.wikipedia.org/wiki/K-means%2B%2B).  
* It works very well in practice, and the `scikit-learn` implementation uses it by default.
* We will explore it in more detail in the next lecture.

## Choosing the right $k$

Generally, we would say that, given some $K$, the $k$-means algorithm "learns" the cluster centers -- that is, the parameters of the model.

But we have not yet considered how to choose the right number of clusters.

That's typically __not__ something one knows in advance.

As an aside:

* This parameter ($K$) is the first example we have seen of a __hyperparameter__.
* A hyperparameter is a parameter that must be set before the model parameters can be learned.

Our basic strategy will be to:

* Iterate through different $K$ and use some criterion to decide which $K$ is most appropriate.
* We will discuss this more in the next lecture.

## Feature Scales

Finally, given the tendency of $k$-means to look for spherical clusters, we should consider the scales of the various features.

In fact, in general when constructing or selecting a distance metric, one needs to think carefully about the scale of the features being used.

## Unscaled Features

For example, consider the case where we are clustering people based on their age, income, and gender.

We might use age in years, income in dollars, and assign gender to the values $\{0, 1\}$.

Thus, the following records:

* Joe Smith, age 27, income USD 75,000, male
* Eve Jones, age 45, income USD 42,000, female

Would be encoded in feature space as:

$$
\begin{bmatrix}27\\75000\\0\end{bmatrix},\begin{bmatrix}45\\42000\\1\end{bmatrix} 
$$

## Unscaled Features, Continued

What would happen if we used Euclidean distance as our dissimilarity metric in this feature space?

(This is what $k$-means uses.)

:::: {.fragment}
Clearly, the influence of income would dominate the other two features.  For
example, a difference of gender is about as significant as a difference of one
dollar of yearly income.

We are unlikely to expose gender-based differences if we cluster using this representation.

The most common way to handle this is __feature scaling.__
::::

## Feature Scaling

The basic idea is to rescale each feature separately, so that its range of values is about the same as all other features.

For example, one may choose to:
    
* shift each feature independently by subtracting the mean over all observed values.
    * This means that each feature is now centered on zero.
* Then rescale each feature so that the standard deviation overall observed values is 1.
    * This means that the feature will have about the same range of values as all the others.

## Feature Scaling Example

For example, let's work with Bortkiewicz's famous
[horse-kick data](http://www.randomservices.org/random/data/HorseKicks.html) which is the the number of soilders in the Prussian cavalry killed by horse kicks over the 20 years between 1875 and 1894, inclusive.

## Ladislaus Bortkiewicz

:::: {.columns}
::: {.column width="40%"}
![Ladislaus Bortkiewicz](figs/Ladislaus_Bortkiewicz.jpg){width=150}^[By no conegut - MacTutor History of Mathematics: http://www-history.mcs.st-andrews.ac.uk/PictDisplay/Bortkiewicz.html, Public Domain, https://commons.wikimedia.org/w/index.php?curid=79219622] ![Law of Small Numbers Book](figs/Bortkiewicz%20LSN%20Book.png){width=200}^[Law of Small Numbers]
:::
::: {.column width="60%"}
* Ladislaus Bortkiewicz (1868 -- 1931)
* Wrote book ["Law of Small Numbers"](https://archive.org/details/dasgesetzderklei00bortrich/mode/2up) in 1898
* Showed how the horse kick data fits the
[Poisson model](03-Probability-and-Statistics-Refresher.html#the-poisson-distribution)
* More generally that rare events in large populations can be statistically modeled using the Poisson distribution
:::
::::

## Feature Scaling Example, Continued

Here is the horse kick data:

In [ ]:
# source: http://www.randomservices.org/random/data/HorseKicks.html
import pandas as pd
df = pd.read_table('data/HorseKicks.txt',index_col='Year',dtype='float')
counts = df.sum(axis=1)
counts

And here is the histogram by year number.

In [ ]:
counts.hist(bins=25,xlabelsize=16);
plt.xlabel('# of Kick Deaths')
plt.ylabel('Count')
plt.title('Histogram of Kick Deaths')
plt.show()

The average:

In [ ]:
counts.mean()

::: {.content-hidden when-profile="web"}
## Feature Scaling Example, Continued
:::

To standardize to zero mean and unit standard deviation, we can use pre-processing
tools from
the [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html)
library.

The distribution after rescaling:

In [ ]:
from sklearn import preprocessing
counts_scaled = pd.DataFrame(preprocessing.scale(counts))
counts_scaled.hist(bins=25,xlabelsize=16);

With a new mean:

In [ ]:
counts_scaled.mean().values

::: {.content-hidden when-profile="web"}
## Feature Scaling Example, Continued
:::

Notice that values that used to be positive have now become negative.

In some situations it may not be sensible to change non-negative values into something else.
It may make more sense to map all values into a fixed range, for example $[0, 1]$.

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
counts_minmax = min_max_scaler.fit_transform(counts.values.reshape(-1,1))
counts_minmax = pd.DataFrame(counts_minmax)
counts_minmax.hist(bins=25,xlabelsize=16);

## Example Application of k-means

Here is a simple example of how $k$-means can be used to reduce
color space and compress data.

Consider the following image.  

* Each color in the image is represented by an integer.  
* Typically we might use 24 bits for each integer (8 bits for R, G, and B).
    
![](figs/L6-annie19980405.jpg){width=350 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Now find $k=16$ clusters of pixels in three dimensional $(R, G, B)$ space
and replace each pixel by its cluster center.

Because there are 16 centroids, we can represent by a 4-bit mapping 
for a compression ratio of $24/4=6\times$.
    
![](figs/L6-annie_016.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Here we cluster into 8 groups (3 bits) for a compression ratio $24/3=8\times$.
    
![](figs/L6-annie_008.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Here we cluster into 4 groups (2 bits) for a compression ratio around $24/2=12\times$.
    
![](figs/L6-annie_004.png){width=500 fig-align="center"}

::: {.content-hidden when-profile="web"}
## Example, Continued
:::

Finally, we use 1 bit (2 color groups) for a compression ratio of $24\times$.
    
![](figs/L6-annie_002.png){width=500 fig-align="center"}

## Recap and Next

Today we covered:

* $k$-means clustering
* strengths and weaknesses
* Importance of initialization and cluster number
* Feature scaling
* Example application

Coming up next, we'll look at some practical aspects of applying $k$-means.